In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import mido
import numpy as np
from mido import Message, MidiFile, MidiTrack
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import sequence

# Define input and output directories
input_folder = "/content/drive/MyDrive/AVISHKAR 22 Sept/Country"
output_folder = "/content/drive/MyDrive/AVISHKAR 22 Sept/Generated with LSTM/Country"

In [ ]:
# List all MIDI files in the input folder
midi_files = [f for f in os.listdir(input_folder) if f.endswith(".mid")]

scaler = MinMaxScaler(feature_range=(0, 1))

n_prev = 30 # Define the number of previous notes to consider

# Initialize X and y for training data
x = []
y = []

# Loop through each MIDI file
for midi_file in midi_files:
    mid = MidiFile(os.path.join(input_folder, midi_file))

    # Preprocess MIDI data
    notes = []
    for msg in mid:
        if not msg.is_meta and msg.channel == 0 and msg.type == "note_on":
            data = msg.bytes()
            notes.append(data[1])

    # Apply scaling and prepare data for training (same as before)
    scaler.fit(np.array(notes).reshape(-1, 1))
    notes = list(scaler.transform(np.array(notes).reshape(-1, 1)))
    notes = [list(note) for note in notes]

    # Append data to training data
    for i in range(len(notes) - n_prev):
        x.append(notes[i : i + n_prev])``
        y.append(notes[i + n_prev])

In [ ]:
#LSTM Model for Generation

model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation("linear"))
optimizer = Adam(learning_rate=0.001)
model.compile(loss="mse", optimizer=optimizer)
filepath = "./Checkpoints/checkpoint_model_{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(
    filepath,
    monitor="val_acc",
    verbose=1,
    save_best_only=False,
    mode="auto",
    save_freq=5,
)

In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

# Train the model on X_train and y_train
model.fit(np.array(x_train), np.array(y_train), batch_size=32, epochs=3, verbose=1, callbacks=[model_save_callback])

Epoch 1/3
   4/1910 [..............................] - ETA: 5:55 - loss: 0.1735
Epoch 1: saving model to ./Checkpoints/checkpoint_model_01.hdf5
   9/1910 [..............................] - ETA: 6:02 - loss: 0.1225
Epoch 1: saving model to ./Checkpoints/checkpoint_model_01.hdf5
  14/1910 [..............................] - ETA: 6:56 - loss: 0.0991
Epoch 1: saving model to ./Checkpoints/checkpoint_model_01.hdf5
  19/1910 [..............................] - ETA: 6:48 - loss: 0.0865
Epoch 1: saving model to ./Checkpoints/checkpoint_model_01.hdf5
  24/1910 [..............................] - ETA: 6:37 - loss: 0.0795
Epoch 1: saving model to ./Checkpoints/checkpoint_model_01.hdf5
  29/1910 [..............................] - ETA: 6:30 - loss: 0.0735
Epoch 1: saving model to ./Checkpoints/checkpoint_model_01.hdf5
  34/1910 [..............................] - ETA: 6:27 - loss: 0.0702
Epoch 1: saving model to ./Checkpoints/checkpoint_model_01.hdf5
  39/1910 [..............................] - ETA: 6:

In [ ]:
X_test = x[-300:]

In [ ]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1, 1)))
prediction = [int(i) for i in prediction]

10/10 [==============================] - 1s 59ms/step


In [ ]:
# Define the output file path
output_file = os.path.join(output_folder, "new1_generated_song.mid")

# Create a new MIDI file for the generated music
mid = MidiFile()
track = MidiTrack()
t = 0

# Add MIDI messages to the track
for note in prediction:
    msg_on = Message.from_dict({'type': 'note_on', 'channel': 0, 'note': note, 'velocity': 67, 'time': 0})
    # You need to add some pauses "note_off"
    msg_off = Message.from_dict({'type': 'note_off', 'channel': 0, 'note': note, 'velocity': 67, 'time': 64})
    track.append(msg_on)
    track.append(msg_off)
    track.append(msg_off)

# Add the track to the MIDI file
mid.tracks.append(track)

# Save the generated MIDI file
mid.save(output_file)